# Conexión por SSH

Inicializar la conexion ssh


In [23]:
import paramiko
import getpass

host = "158.251.88.197"
port = 22
username = "gonzalo"
password = getpass.getpass()

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, port, username, password)

TimeoutError: [WinError 10060] Se produjo un error durante el intento de conexión ya que la parte conectada no respondió adecuadamente tras un periodo de tiempo, o bien se produjo un error en la conexión establecida ya que el host conectado no ha podido responder

`loadBoxes(path,instance)`
--
Se carga las cajas de una instancia

In [6]:
import numpy as np

dimBox = dict()
vol = dict()
volBin = dict() #Volumen de cada bin

def loadBoxes(idClass, boxSize, idInstance):
    boxesinfo = []
    binDim = []
    filePath = '../benchs/class'+str(idClass)+'/'+str(boxSize)+'.txt'
    boxesFile = open(filePath,'r')
    instanceIndex = -1
    countBox = 0
    totalBox = 0
    #Lectura del archivo
    for line in boxesFile.readlines():
        info = line.strip().split()
        
        if(instanceIndex < idInstance and len(info) == 4):
            instanceIndex+=1
            totalBox = int(info[0])
        
        elif(instanceIndex == idInstance):
            #La instancia que se esta buscando
            if(len(info) == 4):
                instanceIndex+=1
                totalBox = int(info[0])
                binDim = [int(info[i]) for i in range(1, len(info))]
                binDim = np.array(binDim)
            
            elif(len(info) == 3 and countBox < totalBox):
                countBox+=1    
                b = np.array([int(info[i]) for i in range(0, len(info))])
                b.sort()
                boxesinfo.append(b)

    countBox = 0
    for boxSize in boxesinfo:
        dimBox[countBox] = np.array(boxSize)
        countBox +=1


Implementé un *servidor MCLP*, el cual escucha instrucciones para generar bins.

El siguiente código lanza el servidor en el background, carga una instancia y lo deja escuchando peticiones por el puerto 8080

In [7]:
def loadInstance(idClass=1, boxSize=100, idInstance=0, port=8060):
    transport = ssh.get_transport()
    channel = transport.open_session()
    loadBoxes(idClass, boxSize, idInstance)
    channel.exec_command("killall BASIC_BSG_MCLP; ./mclp/Metasolver/BASIC_BSG_MCLP mclp/Metasolver/problems/mclp/benchs/class"+ \
                         str(idClass)+"/"+str(boxSize)+".txt -i "+str(idInstance)+" --port="+str(port))
    return dimBox, channel

boxes, channel = loadInstance(1,100,0,port)   

Para cerrar el servidor MCLP

In [8]:
def close_mclp_server(port=8060):
    stdin, stdout, stderr = ssh.exec_command("echo END | netcat localhost "+str(port))
    print(stdout.readlines()[0])
    

# Multiple Container Loading Problem

In [25]:
def mclp(idClass, idInstance, boxSize):
    global channel
    swapIter = 10
    #___, channel = loadInstance(idClass,boxSize, idInstance,port)
    print(port)
    bins = solve(swapIter)
    return len(bins)

## VARIABLES GLOBALES
idClass = 1
boxSize = 100
idInstance = 0
port = 8060
global channel

i = mclp(idClass, idInstance, boxSize)

AttributeError: 'NoneType' object has no attribute 'open_session'

# Metodos implementados

### Métodos de BPP


In [10]:
def solve(swapIter):
    cantBoxes = len(dimBox)
    totalBoxes = boxes
    
    #Llamada a BSG
    bins = generate_bins()
    
    firstSol = len(bins)
    
    for i in range(swapIter):
        toReduce, bins = popRandomBin(bins) 
        bins = reduceBins(bins, toReduce)
    
    return len(bins)


**Generación de conjunto inicial de bins con BSG**

La siguiente función genera la lista inicial de bins. Cada línea representa un bin indicando el porcentaje de llenado y los ids de sus cajas.

Es necesario modificarla para que en vez de imprimir el contenido de los bins, retorne una estructura adecuada.

In [11]:
def generate_bins(port=8060):
    #probarlo con distintas instancias
    stdin, stdout, stderr = ssh.exec_command("echo generate_bins | netcat localhost "+str(port))
    flag = False
    bins = []
    count = 0
    for line in stdout.readlines():
        if "BINS:" in line: 
            flag = True
            continue
        if flag:    
            #Modificar igual que generate_bin
            line = line.strip().split()
            vol = float(line[0]) #Porcentaje de llenado del bin
            newBin = np.array([int(line[x]) for x in range(1,len(line))])
            #calcular el volumen
            bins.append(newBin)
            volBin[count] = newBin
            count +=1
    #recordar forma del arreglo        
    return bins

**Construcción de bin priorizando cajas (BSG)**

In [21]:
import paramiko
import getpass
import numpy as np

def generate_bin(boxes, new_box, port=8060):
    global channel
    index = 0
    leftboxes = []
    newBin = []
    boxes = np.insert(boxes, 1, new_box)
    boxes_str = [str(box) for box in boxes]
    pboxes_str = str(new_box)
    
    print("echo generate_bin " + " ".join(boxes_str) + " -1 " + pboxes_str +  " -2 | netcat localhost "+str(port))
    stdin, stdout, stderr = ssh.exec_command("echo generate_bin " + " ".join(boxes_str) + " -1 " + pboxes_str +  " -2 | netcat localhost "+str(port))
    lines = stdout.readlines()

    while( len(lines) == 0 ):
        ___, channel = loadInstance(idClass, boxSize, idInstance, port)
        stdin, stdout, stderr = ssh.exec_command("echo generate_bin " + " ".join(boxes_str) + " -1 " + pboxes_str +  " -2 | netcat localhost "+str(port))
        lines = stdout.readlines()
        
    
    for line in lines: 
        #ajustar linea para almacenar los conjuntos de cajas
        line = line.strip().split()
        
        if(index == 0):
            #primera linea boxes 
            percent = float(line.pop(0))
            newBin = [int(x) for x in line]
            
        else:
            #segunda linea left boxes
            leftboxes = [int(x) for x in line]
        index+=1

    print("NEW BIN")
    print(newBin)
    print("LEFT BOXES")
    print(leftboxes)
    return newBin, leftboxes

testBin, testLeft = generate_bin([1,2,3,4,5],8)
print(testBin)

echo generate_bin 1 8 2 3 4 5 -1 8 -2 | netcat localhost 8060
NEW BIN
[]
LEFT BOXES
[]
[]
Socket exception: Se ha forzado la interrupción de una conexión existente por el host remoto (10054)
Socket exception: Se ha forzado la interrupción de una conexión existente por el host remoto (10054)


Faltaría:

* Cargar la instancia en python (cajas con sus dimensiones) (Listo)
* Implementar algoritmo. (Listo)

A grandes rasgos el algoritmo hace los siguiente:
1. Generación de bins iniciales usando **BSG**
2. Selección de bin a desarmar y almacenar cajas en $C$
3. Mientras $C$ no quede vacío o máximo de iteraciones:

   1. Seleccionar caja $c$ de $C$
   2. Seleccionar bin de destino $B$
   3. Usar **BSG** para generar bin $B'$ usando cajas $B \cup \{c\}$, priorizando $c$. Es posible que **BSG** retorne conjunto de cajas residuales $R$
   4. Si $R$ es mejor que $c$, $B$ se reemplaza por $B'$ en el conjunto de bins y $C \gets C \cup R$
   
 4. Volver a 2 (seleccionar otro bin para desarmar)

In [ ]:
import copy as cp

def reduceBins(bins, toReduce):

    lastBins = cp.deepcopy(bins)
    breakedBin = toReduce
    iterator = 0

    while(len(breakedBin) == 0 or iterator <100):

        pivBin, bins = popRandomBin(bins)    
        complexBox, evalComplexBox = getMostComplexBox(breakedBin)
        #Metodo de llamada al BSG
        boxes, leftBoxes = generate_bin(pivBin, complexBox)
        
        #Se calcula la complejidad de la caja más compleja
        dimComplexBox = dimBox[complexBox]
        
        evalComplexBox = calculateComplex(complexBox, dimComplexBox)

        if( getMostComplexBox(leftBoxes)[1] < evalComplexBox and vol(leftBoxes) <= vol([complexBox]) ):
            
            #Se inserta el nuevo bin dentro del conjunto de bins
            bins.append(boxes)
            
            #Se elimina (desde el conjunto de cajas a repartir) la caja insertada en el nuevo bin 
            index = np.where(breakedBin == complexBox)
            breakedBin = np.delete(breakedBin, index[0])
            
            #Se agregan las cajas que se cambiaron
            breakedBin += leftBoxes
        else:

            bins.append(pivBin)

    #Regresar al estado anterior si 
    if(len(breakedBin) > 0):
        return lastBins
    #Sino
    else: 
        return bins
        

## Getter - Setter - POP

In [ ]:
import random as rand

def popRandomBin(bins):
    rPos = int(rand.random()*len(bins))
    theBin = bins.pop(rPos)
    return theBin, bins


testBin = [1,2,3,4,5,6]
___, testBin = popRandomBin(testBin)
print(len(testBin))

In [ ]:
def getMostComplexBox(bin):
    idBox = None
    eval = 0
    
    for b in bin:
        boxShape = dimBox[b] #Cargar instancia
        currentEval = calculateComplex(b,boxShape)    
        if (idBox is None):
            idBox = b
            eval = currentEval
        else:
            if(currentEval > eval):
                idBox = b
                eval = currentEval
    return (idBox, eval)

## Extras 

In [ ]:
def calculateComplex(idBox, dim):
    #Revisar metodo sort
    x,y,z = dim
    expo = 1
    vol = x*y*z

    return ( ( (x/z)*(x/y)*(y/z) )**expo ) * vol

In [ ]:
def vol(listBoxes):
    totalVol = 0
    for box in listBoxes:
        x,y,z = dimBox[box]
        totalVol += x*y*z
    return totalVol

testBin = [14,5,1,3]
print(vol(testBin))